In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service #Chromedriver.exe is required for the application to work properly!
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import matplotlib.pyplot as plt
import numpy as np
import re, io, json, os 
import networkx as nx
import networkit as nk
import pandas as pd

### Functions

In [2]:
def get_all_links(driver):
    links = []
    webdom = ["fizyka.pw.edu","if.pw.edu"] #List of substrings which must be included in our URL, for instance only certain domain description. 
    elements = driver.find_elements(By.TAG_NAME,'a')
    for elem in elements:
        href = elem.get_attribute("href")
        if href == None: continue
        types = [] 
        if ((webdom[0] in href or webdom[1] in href) and (".pdf" not in href) and (".jpg" not in href)):
            links.append(href)
    links = np.array(links)
    links = links[links != np.array(None)]
    links = np.unique(links)
    return links

### Driver

In [3]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-notifications')
service = Service('./chromedriver.exe')
driver = webdriver.Chrome(service = service, options=options)
driver.maximize_window()

### Link extraction within single pages

In [ ]:
global_dict = {}
supp = []
DF = pd.read_csv("WWW.csv", encoding = "ISO-8859-1") #WWW.csv - file containing webpages URLs
my_all_links = list(DF["URL"].astype(str))

# my_all_links = [] #Alternative direct list of certain websites
for i,link in enumerate(my_all_links):  
    if not "http" in link: link = "https://" + link
    driver.get(link)
    curr = driver.current_url
    print(f"CURR: {curr} ----> OK")
    try:
        global_dict[curr] = get_all_links(driver)
    except:
        continue

### Link Extraction

In [ ]:
print("=======================================================================================")
print("============================Extracting links within domain=============================")
print("=======================================================================================")

cnt = 0
while True:
    glob_key = list(global_dict.keys())[cnt]
    glob = global_dict[glob_key]
    if len(glob) == 0: 
        cnt += 1
        continue
    else:
        if "if.pw" in glob_key:
            m = re.search('~(.+?)/', glob_key)
            mm = m[0][1:-1]
        elif "fizyka.pw" in glob_key:
            mm = glob_key[:glob_key.index("fizyka")-1]
        else:
            mm = glob_key
        for i in glob:
            if mm in i:
                try:
                    driver.get(i)
                    global_dict[glob_key] = np.append(global_dict[glob_key],get_all_links(driver))
                except: continue
            else: continue  
        global_dict[glob_key] = list(filter(lambda x: mm not in x,global_dict[glob_key]))
        global_dict[glob_key] = np.array(global_dict[glob_key])
        global_dict[glob_key] = np.unique(global_dict[glob_key])
        cnt += 1
        print("==========")
    if cnt == len(global_dict): break

### Data to JSON

In [9]:
for i in global_dict:
    global_dict[i] = global_dict[i].tolist()

with open(f"data.json","w") as f:
    json.dump(global_dict,f,indent=6) 

### Data from JSON & DataFrame

In [ ]:
with open("data.json","r") as f:
    XC = json.load(f)

Big_joe = []  
for i,j in XC.items():
    for k in j:
        Big_joe.append([i,k])

XX = pd.DataFrame(Big_joe, columns=["WWW1","WWW2"])
XX.to_csv("data.csv")

### Network creation and visualization
This method is quite primitive. You'd better use some other special programs like Cytoscape, GePhi etc. to create nice visualization.

In [ ]:
G = nx.Graph()
G.add_edges_from(XX.values)
list_degree=list(G.degree()) 
nodes , degree = map(list, zip(*list_degree)) 
plt.figure(figsize=(10,10))
nx.draw(G, nodelist=nodes, with_labels = True, node_size=[(v * 20)+1 for v in degree], node_color = "yellow", pos=nx.nx_agraph.graphviz_layout(G,"fdp"))
plt.show()